In [2]:
import pickle
import os
import glob
import numpy as np
# import model to load parameters
from model import Endo3D
import torch
import torch.nn as nn
from torch.utils import data
import json
from sequence_loder import sequence_loder

In [3]:
# import sequence_loder from yitong's path
# import sys
# # insert at 1, 0 is the script path (or '' in REPL)
# sys.path.insert(1, '/home/yitong/venv_yitong/sacro_wf_analysis/utils')
# from sequence_loder import sequence_loder

In [4]:
paramPath = "/home/yitong/venv_yitong/sacro_wf_analysis/params"

In [5]:
os.listdir(paramPath)

['params_c3d_1200.pkl',
 'cross_validation',
 '.pkl',
 'params_endo3d_save_point.pkl']

In [6]:
params = pickle.load(open(paramPath + "/params_endo3d_save_point.pkl", 'rb'))

In [7]:
params

119547037146038801333356

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
vanilla_net = Endo3D().to(device)
vanilla_net.load_state_dict(torch.load(paramPath + "/params_endo3d_save_point.pkl"))

<All keys matched successfully>

In [10]:
vanilla_net

Endo3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv5b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (fc6): Linear(in_features=8192, out_features=4096, bias=True)
  (fc7): Linear(in_features=4096, out_features=4096, bias=True)
  (fc8): Linear(in_features=4096, out_features=1200, bias=True)
  (fc_phase): Linear(in_features=1200, out_features=6, bias=True)
  (relu): ReLU()
  (pool_122

In [11]:
# get predicted labels, true labels and f1 scores for the given video name
def sequence_maker(model, video_name, div, device):
    sacro = sequence_loder(div, device)
    sacro.whole_len_output(video_name)
    whole_loder = data.DataLoader(sacro, 20)
    seq_pre = []
    seq_true = []
    fc_list = []
    softmax_vec_list = []
    for labels_val, inputs_val in whole_loder:
        inputs_val = inputs_val.float().to(device)
        labels_val = labels_val.long().to(device)
        output, x_fc = model.forward_cov(inputs_val)
        _, predicted_labels = torch.max(output.cpu().data, 1)
        for i in range(predicted_labels.numpy().shape[0]):
            seq_pre.append(predicted_labels.numpy()[i])
            seq_true.append(labels_val.cpu().numpy()[i])
            softmax_vec_list.append(output.cpu()[i])
            fc_list.append(x_fc[i, :])
    return seq_pre, seq_true, softmax_vec_list



In [12]:
os.getcwd()

'/home/jenna/jennaCode'

In [22]:
# use the first training video to test whether the nn is working
path = '/home/yitong/venv_yitong/sacro_wf_analysis/data/'
whole_path = path+'sacro_sequence/whole'
videos_path = path+'sacro_jpg'
print(videos_path)

folders = ['folder1']
div = ['div1', 'div2', 'div3', 'div4', 'div5', 'div6', 'div7']

for counter, folder in enumerate(folders):
    print('The current folder is:' + folder)
    whole_folder_path = os.path.join(whole_path, folder)
    with open(os.path.join(videos_path, 'dataset_' + div[counter] + '.json'), 'r') as json_data:
        temp = json.load(json_data)
    # get the training videos
    video_list = temp['train']
    video_list = [video_list[0]]
    
    for video_name in video_list:
        save_path = os.path.join(whole_folder_path, video_name)

        with torch.no_grad():
            seq_pred, seq_true, softmax_vects = sequence_maker(vanilla_net, video_name, div[counter], device)
            print("The current video is {}".format(video_name))
            print("The output list of softmax layer is:")
            print(softmax_vects)

/home/yitong/venv_yitong/sacro_wf_analysis/data/sacro_jpg
The current folder is:folder1


100%|███████████████████████████████████████| 1109/1109 [01:02<00:00, 17.79it/s]


The current video is e8d3eabc-edd1-414f-9d95-277df742655a
The output list of softmax layer is:
[tensor([-4.7431, -3.3514, -0.0625, -5.7537, -7.2889, -4.3433]), tensor([-6.0874, -2.7801, -0.0675, -8.2384, -7.5010, -8.7425]), tensor([-1.2796e+01, -3.9333e+00, -1.9776e-02, -2.2588e+01, -1.6675e+01,
        -1.6094e+01]), tensor([-8.2164e+00, -6.0368e+00, -2.6733e-03, -1.1669e+01, -1.3736e+01,
        -1.4162e+01]), tensor([-4.4792, -3.9489, -0.0393, -6.6161, -6.3073, -5.3404]), tensor([-3.3746, -2.3141, -0.1628, -5.5073, -5.0475, -5.0047]), tensor([-7.6849e+00, -8.7468e+00, -7.0344e-04, -9.8763e+00, -1.0757e+01,
        -1.1346e+01]), tensor([ -5.4972,  -3.7018,  -0.0300, -10.4030,  -9.2251,  -7.3588]), tensor([-8.8061e+00, -4.1779e+00, -1.5606e-02, -1.8725e+01, -1.8823e+01,
        -1.2529e+01]), tensor([-4.8563, -3.0261, -0.0592, -7.6477, -8.5138, -7.4798]), tensor([ -7.2855,  -3.6743,  -0.0266, -10.4004,  -8.6474, -10.4167]), tensor([-1.9755, -2.4562, -0.8678, -2.1473, -1.7517, -2.7276

In [13]:
#yay, it's working! now change it to train on all the training data assigned to each fold/division and get the softmax as output vectors

In [38]:
prob_vec_list = []
for tensor in softmax_vects:
    prob_vec = torch.exp(tensor)
    prob_vec_list.append(prob_vec.numpy())
prob_vec_list = np.array(prob_vec_list)

In [39]:
prob_vec_list[:100]

array([[8.71187076e-03, 3.50338221e-02, 9.39406931e-01, 3.17101646e-03,
        6.83073536e-04, 1.29932892e-02],
       [2.27131858e-03, 6.20344132e-02, 9.34717655e-01, 2.64314003e-04,
        5.52532671e-04, 1.59649178e-04],
       [2.77266918e-06, 1.95786040e-02, 9.80418503e-01, 1.54988924e-10,
        5.73059182e-08, 1.02475390e-07],
       [2.70184915e-04, 2.38909083e-03, 9.97330308e-01, 8.55402686e-06,
        1.08323104e-06, 7.06943979e-07],
       [1.13424743e-02, 1.92762576e-02, 9.61425543e-01, 1.33862824e-03,
        1.82297186e-03, 4.79416410e-03],
       [3.42329517e-02, 9.88520011e-02, 8.49726081e-01, 4.05721739e-03,
        6.42513065e-03, 6.70663314e-03],
       [4.59705654e-04, 1.58965966e-04, 9.99296784e-01, 5.13756313e-05,
        2.13000803e-05, 1.18130483e-05],
       [4.09820303e-03, 2.46794019e-02, 9.70456660e-01, 3.03417019e-05,
        9.85303122e-05, 6.36939949e-04],
       [1.49809421e-04, 1.53310141e-02, 9.84515429e-01, 7.37719308e-09,
        6.68638434e-09, 

In [17]:
save_path = os.path.join(save_path_prefix, "abc")
# create the path if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [19]:
save_path_prefix = '/home/jenna/jennaCode/prob_vectors/'
save_path = os.path.join(save_path_prefix, 'abc')
file = open(os.path.join(save_path, 'prob_vectors.pickle'), 'wb+')
file.close()